**NAME:** Eric Graham



# In Class Assignment 3
In the following assignment you will be asked to fill in python code and derivations for a number of different problems. Please read all instructions carefully.


### Downloading the Document Data
Please run the following code to read in the "20 newsgroups" dataset from sklearn's data loading module.

In [1]:
from sklearn.datasets import fetch_20newsgroups_vectorized
import numpy as np

# this takes about 30 seconds to compute, read the next section while this downloads
ds = fetch_20newsgroups_vectorized(subset='train')

# this holds the continuous feature data (which is tfidf)
print('features shape:', ds.data.shape) # there are ~11000 instances and ~130k features per instance
print('target shape:', ds.target.shape)
print('range of target:', np.min(ds.target),np.max(ds.target))
print('Data type is', type(ds.data), float(ds.data.nnz)/(ds.data.shape[0]*ds.data.shape[1])*100, '% of the data is non-zero')

features shape: (11314, 130107)
target shape: (11314,)
range of target: 0 19
Data type is <class 'scipy.sparse._csr.csr_matrix'> 0.1214353154362896 % of the data is non-zero


## Understanding the Dataset
Look at the description for the 20 newsgroups dataset at http://qwone.com/~jason/20Newsgroups/. You have just downloaded the "vectorized" version of the dataset, which means all the words inside the articles have gone through a transformation that binned them into 130 thousand features related to the words in them.  

**Question 1**:

a) How many instances are in the dataset?

**11314**

b) What does each instance represent?

**Each instance represents a newsgroup post (numerically vectorized)**

c) How many classes are in the dataset and what does each class represent?

**20 news groups, each for a different topic**

d) Would you expect a classifier trained on this data would generalize to documents written in the past week? Why or why not?

**I don't think it would generalize well because the dataset was last updated in 2008 (or prior), so topics of discussion and semantic style have likely changed since then.**

e) Is the data represented as a sparse or dense matrix?

**Sparse: only .12% of the data is non-zero**

## Measures of Distance
In the following block of code, we isolate three instances from the dataset. The instance "`a`" is from the group *computer graphics*, "`b`" is from from the group *recreation autos*, and "`c`" is from group *recreation motorcycle*.

**Question 2**:

For each pair of instances calculate the (1) Euclidean distance, and (2) Jaccard similarity.
Which distance seems more appropriate to use for this data? **Explain your reasoning.**


*Note: Remember that the Jaccard similarity is only for binary valued vectors, so convert vectors to binary using a threshold.*

In [2]:
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import jaccard
import numpy as np

# get first instance (comp)
idx = 550
a = ds.data[idx].todense()
a_class = ds.target_names[ds.target[idx]]
print('Instance A is from class', a_class)

# get second instance (autos)
idx = 4000
b = ds.data[idx].todense()
b_class = ds.target_names[ds.target[idx]]
print('Instance B is from class', b_class)

# get third instance (motorcycle)
idx = 7000
c = ds.data[idx].todense()
c_class = ds.target_names[ds.target[idx]]
print('Instance C is from class', c_class)

# Enter distance comparison below for each pair of vectors:

# Flatten to 1D arrays
a_flat = np.asarray(a).flatten()
b_flat = np.asarray(b).flatten()
c_flat = np.asarray(c).flatten()

# Euclidean Distance calculations
ab_euclidean = euclidean(a_flat, b_flat)
ac_euclidean = euclidean(a_flat, c_flat)
bc_euclidean = euclidean(b_flat, c_flat)

# Convert to binary for Jaccard (threshold at 0)
a_binary = (a_flat > 0).astype(int)
b_binary = (b_flat > 0).astype(int)
c_binary = (c_flat > 0).astype(int)

# Jaccard Distance calculations
ab_jaccard = jaccard(a_binary, b_binary)
ac_jaccard = jaccard(a_binary, c_binary)
bc_jaccard = jaccard(b_binary, c_binary)

print('\n\nEuclidean Distance\n ab:', ab_euclidean, 'ac:', ac_euclidean, 'bc:', bc_euclidean)
print('Jaccard Distance (vectors should be boolean values)\n ab:', ab_jaccard, 'ac:', ac_jaccard, 'bc:', bc_jaccard)

p = 'Jaccard distance is more appropriate for this data because it focuses on the presence/absence of features (words) rather than their magnitude. It ignores the many zero values in sparse vectors, making it more meaningful for this text data. Euclidean is entirely based on magnitude or scale, so it would be biased in this case.'

print('\n\nThe most appropriate distance is...because...')
print(p)

Instance A is from class comp.graphics
Instance B is from class rec.autos
Instance C is from class rec.motorcycles


Euclidean Distance
 ab: 1.0985184671870858 ac: 1.1891405425398234 bc: 0.9177794226661624
Jaccard Distance (vectors should be boolean values)
 ab: 0.8821138211382114 ac: 0.8754716981132076 bc: 0.9087947882736156


The most appropriate distance is...because...
Jaccard distance is more appropriate for this data because it focuses on the presence/absence of features (words) rather than their magnitude. It ignores the many zero values in sparse vectors, making it more meaningful for this text data. Euclidean is entirely based on magnitude or scale, so it would be biased in this case.


## Using scikit-learn with KNN
Now let's use stratified cross validation with a holdout set to train a KNN model in `scikit-learn`. Use the example below to train a KNN classifier. The documentation for `KNeighborsClassifier` is here: http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html  

**Question 3**:
Use the code below to test what value of `n_neighbors` works best for the given data.
What is the accuracy of the best classifier you can create for this data (by changing only the `n_neighbors` parameter)?

 *Note: do NOT change the metric to be anything other than `'euclidean'`. Other distance functions are not optimized for the amount of data we are working with.*

In [3]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


cv = StratifiedShuffleSplit(n_splits=1, test_size = 0.5, train_size=0.5)

# fill in the training and testing data and save as separate variables
for trainidx, testidx in cv.split(ds.data,ds.target):
    # note that these are sparse matrices
    X_train = ds.data[trainidx]
    X_test = ds.data[testidx]
    y_train = ds.target[trainidx]
    y_test = ds.target[testidx]

# fill in your code  here to train and test
# calculate the accuracy and print it for various values of K

# Looping over values of K
for K in [1, 2, 3, 4, 5, 7, 9, 11, 15, 20]:
    clf = KNeighborsClassifier(n_neighbors=K, weights='uniform', metric='euclidean')

    # Train classifier
    clf.fit(X_train, y_train)

    # Predict
    y_pred = clf.predict(X_test)

    # Calculate accuracy
    acc = accuracy_score(y_test, y_pred)
    
    print('Accuracy of classifier with %d neighbors is: %.2f' % (K, acc))

Accuracy of classifier with 1 neighbors is: 0.60
Accuracy of classifier with 2 neighbors is: 0.54
Accuracy of classifier with 3 neighbors is: 0.52
Accuracy of classifier with 4 neighbors is: 0.50
Accuracy of classifier with 5 neighbors is: 0.49
Accuracy of classifier with 7 neighbors is: 0.47
Accuracy of classifier with 9 neighbors is: 0.46
Accuracy of classifier with 11 neighbors is: 0.45
Accuracy of classifier with 15 neighbors is: 0.43
Accuracy of classifier with 20 neighbors is: 0.42


**The best accuracy is .59 with 1 neighbor**

**Question 4**: With sparse data, does the use of a KDTree representation make sense? Why or Why not?

Enter your answer below:

**No, it partitions data along individual features, so we see the curse of dimensionality because we have ~130k features and since most feature values are zero it can't make meaningful separations.**

_____
## KNN extensions - Centroids
Now lets look at a very closely related classifier to KNN, called nearest centroid. In this classifier (which is more appropriate for big data scenarios and sparse data), the training step is used to calculate the centroids for each class. These centroids are saved. Unknown attributes, at prediction time, only need to have distances calculated for each saved centroid, drastically decreasing the time required for a prediction.

**Question 5**: Use the template code below to create a nearest centroid classifier. Test which metric has the best cross validated performance: Euclidean or Manhattan. In `scikit-learn` you can see the documentation for NearestCentroid here:
- http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestCentroid.html#sklearn.neighbors.NearestCentroid

and for supported distance metrics here:
- http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.distance_metrics.html#sklearn.metrics.pairwise.distance_metrics

In [4]:
from sklearn.neighbors import NearestCentroid

clf = NearestCentroid(metric='euclidean')

# Euclidean
clf.fit(X_train, y_train)
y_pred_euclidean = clf.predict(X_test)
acc_euclidean = accuracy_score(y_test, y_pred_euclidean)

# Manhattan
clf_manhattan = NearestCentroid(metric='manhattan')
clf_manhattan.fit(X_train, y_train)
y_pred_manhattan = clf_manhattan.predict(X_test)
acc_manhattan = accuracy_score(y_test, y_pred_manhattan)

print('Euclidean accuracy:', acc_euclidean)
print('Manhattan accuracy:', acc_manhattan)

p = 'Euclidean. It has the highest accuracy, likely because the squared sum of distances reflects meaningful differences in the data.'

print('The best distance metric is: ', p)

/home/eric/miniconda3/envs/jupyter/lib/python3.11/site-packages/sklearn/neighbors/_nearest_centroid.py:244: UserWarning: self.within_class_std_dev_ has at least 1 zero standard deviation.Inputs within the same classes for at least 1 feature are identical.
  warnings.warn(
/home/eric/miniconda3/envs/jupyter/lib/python3.11/site-packages/sklearn/neighbors/_nearest_centroid.py:244: UserWarning: self.within_class_std_dev_ has at least 1 zero standard deviation.Inputs within the same classes for at least 1 feature are identical.
  warnings.warn(


Euclidean accuracy: 0.4187732013434683
Manhattan accuracy: 0.17588828000707088
The best distance metric is:  Euclidean. It has the highest accuracy, likely because the squared sum of distances reflects meaningful differences in the data.


## Naive Bayes Classification
Now let's look at the use of the Naive Bayes classifier. The 20 newsgroups dataset has 20 classes and about 130,000 features per instance. Recall that the Naive Bayes classifer calculates a posterior distribution for each possible class. Each posterior distribution is a multiplication of many conditional distributions:

$${\arg \max}_{j} \left(p(class=j)\prod_{i} p(attribute=i|class=j) \right)$$

where $p(class=j)$ is the prior and $p(attribute=i|class=j)$ is the conditional probability.

**Question 6**:

With this many classes and features, how many different conditional probabilities need to be parameterized? How many priors need to be parameterized?

Enter you answer here:

**With 20 classes and 130,000 features, 2,600,000 conditional probabilities need to be parameterized and we need 20 prior probabilities (one per class) to see how common each class is in the training data."**

In [5]:
# Use this space for any calculations you might want to do

___
## Naive Bayes in Scikit-learn
Scikit has several implementations of the Naive Bayes classifier: `GaussianNB`, `MultinomialNB`, and `BernoulliNB`. Look at the documentation here: http://scikit-learn.org/stable/modules/naive_bayes.html Take a look at each implementation and then answer this question:

**Question 7**:

a) If the instances contain mostly continuous attributes, would it be better to use Gaussian Naive Bayes, Multinomial Naive Bayes, or Bernoulli? And Why?

**Gaussian: it assumes that features follow a normal distribution, while multinomial is intended for discrete data and Bernoulli is designed for binary data."**

b) What if the data is sparse, does this change your answer? Why or Why not?

**Sparse data by definition has so many zero-value features that it wouldn't meet the assumption of normality, in which case we could use multinomial or Bernoulli (with a threshold to binarize the features)."

## Naive Bayes Comparison
For the final section of this notebook let's compare the performance of Naive Bayes for document classification. Look at the parameters for `MultinomialNB`, and `BernoulliNB` (especially `alpha` and `binarize`).

**Question 8 (Extra credit)**:

a) Using the example code below, change the parameters for each classifier and see how accurate you can make the classifiers on the test set.

**See below**

b) Why are these implementations so fast to train? What does the `'alpha'` value control in these models (*i.e.*, how does it change the parameterizations)?

**Alpha adds a smoothing parameter to each feature value to prevent zero probabilities. Higher alphas will have more uniform probabilities, while lower alphas are truer to the observations.**

In [6]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB


clf_mnb = MultinomialNB(alpha=1.0)
clf_bnb = BernoulliNB(alpha=1.0, binarize=0.0)

# Test MultinomialNB
clf_mnb.fit(X_train, y_train)
y_pred_mnb = clf_mnb.predict(X_test)
acc_mnb = accuracy_score(y_test, y_pred_mnb)

# Test BernoulliNB
clf_bnb.fit(X_train, y_train)
y_pred_bnb = clf_bnb.predict(X_test)
acc_bnb = accuracy_score(y_test, y_pred_bnb)


print('MultinomialNB accuracy:', acc_mnb)
print('BernoulliNB accuracy:', acc_bnb)

p1 = 'they only need to calculate simple probability statistics (counts and frequencies) from the training data, requiring just one pass through the data with no iterative optimization.'

p2 = 'smoothing (Laplace/additive smoothing) - it adds alpha to all feature counts to prevent zero probabilities and handle unseen features, with higher alpha values providing stronger smoothing.'

print('These classifiers are so fast because...', p1)
print('The alpha values control...', p2)

MultinomialNB accuracy: 0.7090330563903129
BernoulliNB accuracy: 0.6213540745978434
These classifiers are so fast because... they only need to calculate simple probability statistics (counts and frequencies) from the training data, requiring just one pass through the data with no iterative optimization.
The alpha values control... smoothing (Laplace/additive smoothing) - it adds alpha to all feature counts to prevent zero probabilities and handle unseen features, with higher alpha values providing stronger smoothing.


That's all! Please **upload your rendered notebook** and please include **your  name** in the notebook submission.